# Impacto de BERT y análisis de sentimiento en el modelo OLS

## Importación de datos y limpieza del *dataframe*.

In [1]:
import pandas as pd #permite manipular los dataframes, que serían el equivalente a los Excel
import numpy as np #en caso se necesiten cálculos numéricos, como p. ej. encontrar valores nulos 
import statsmodels.api as sm #para elaborar modelos OLS

base=pd.read_csv('data/clases/habitaclia4043_1y2.csv') #aquí embedding BERT
base2=pd.read_csv('data/clases/habitaclia4043_3.csv') #aquí el análisis de sentimiento

In [2]:
print(base2.columns.tolist())

['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'OBJECTID_1', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun', 'precio_eur', 'superficie', 'superficie_2', 'Unit_price', 'Ln_total_pr', 'Ln_unit_pr', 'numero_habitaciones', 'numero_bano', 'ratio_bano_hab', 'numero_aseo', 'ascensor', 'interac_planta', 'numero_de_piso', 'anyo_constr', 'anyo_constr_ponderad', 'antig_ponderad', 'Inverse_Age', 'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007', 'superficie_terraza_m2', 'grand_terr_20m2', 'superficie_jardin_m2', 'superficie_salon', 'bool_despacho', 'bool_buhardilla', 'bool_trastero', 'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado', 'bool_ascensor', 'descripcion', 'bool_aire_acondicionado', 'bool_calefaccion', 'bool_chimenea', 'texto_destacado', 'Description', 'calificacion_consumo_letra', 'calificacion_consumo_valor', 'calificacion_emision_letra', 'calificacion_emision_valor', 'Du

In [3]:
print(base.columns.tolist())

['Unnamed: 0.1', 'Unnamed: 0', 'OBJECTID_1', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun', 'precio_eur', 'superficie', 'superficie_2', 'Unit_price', 'Ln_total_pr', 'Ln_unit_pr', 'numero_habitaciones', 'numero_bano', 'ratio_bano_hab', 'numero_aseo', 'ascensor', 'interac_planta', 'numero_de_piso', 'anyo_constr', 'anyo_constr_ponderad', 'antig_ponderad', 'Inverse_Age', 'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007', 'superficie_terraza_m2', 'grand_terr_20m2', 'superficie_jardin_m2', 'superficie_salon', 'bool_despacho', 'bool_buhardilla', 'bool_trastero', 'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado', 'bool_ascensor', 'descripcion', 'bool_aire_acondicionado', 'bool_calefaccion', 'bool_chimenea', 'texto_destacado', 'Description', 'calificacion_consumo_letra', 'calificacion_consumo_valor', 'calificacion_emision_letra', 'calificacion_emision_valor', 'Dum_EPC', 'EPC_A_e

In [4]:
base_columns=['EPC_A_emision', 'EPC_B_emision','EPC_C_emision','EPC_D_emision','EPC_E_emision','EPC_F_emision',
    'superficie', 'bool_ascensor', 'interac_planta','bool_piscina_comunitaria',
    'Precio_red','Year_1982_2006','Year_After_2007','Estado_contemporaneidad_calidad',
    'Ausencia_singulares_presencia_arm_cocina','bool_calefaccion',
    'bool_aire_acondicionado', 'ratio_bano_hab','grand_terr_20m2',
    'H_tamaviv','FIRE_pr','desplaz_ponderado','Income_Household_2016', 'DP2e',
    'CT_renta_meda_CPA','dum_mar_200m','Dist_CBD2','opin_ruido_si_pr','Ln_total_pr',
    'OBJECTID_1']

base = base[base_columns + ['mpBERT']].sort_index(axis=1)
base2 = base2[['OBJECTID_1', 'ACP_NEU_noPOS', 'ACP_NEG']].sort_index(axis=1)

df = pd.merge(base, base2, on='OBJECTID_1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4043 entries, 0 to 4042
Data columns (total 33 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Ausencia_singulares_presencia_arm_cocina  3859 non-null   float64
 1   CT_renta_meda_CPA                         4043 non-null   float64
 2   DP2e                                      4043 non-null   float64
 3   Dist_CBD2                                 4043 non-null   float64
 4   EPC_A_emision                             4043 non-null   float64
 5   EPC_B_emision                             4043 non-null   float64
 6   EPC_C_emision                             4043 non-null   float64
 7   EPC_D_emision                             4043 non-null   float64
 8   EPC_E_emision                             4043 non-null   float64
 9   EPC_F_emision                             4043 non-null   float64
 10  Estado_contemporaneidad_calidad     

Ahora elimino `OBJECTID_1`, dado que la incluí sólo para hacer un match entre dataframes. Ya no la necesito.

In [5]:
df=df.drop('OBJECTID_1',axis=1) #axis 1 son las columnas

Hago el **filtro de outliers**.

In [7]:
df = df[(df['Ln_total_pr'] >= 11.2842035426135) & (df['Ln_total_pr'] <= 13.8409882055083)]

En caso haya filas vacías, las elimino.

In [9]:
#Para no sobreescribir, creo una nueva variable: 'df'
df = df.replace([np.inf, -np.inf], np.nan).dropna()

Reviso las **correlaciones**.

In [71]:
# Suponiendo que 'base' es tu dataframe y 'variable1' y 'variable2' son las variables que quieres comparar
variable1 = df['mpBERT']
variable2 = df['Ln_total_pr']

# Calcular la correlación y el p-valor
correlacion, p_valor = pearsonr(variable1, variable2)

print("Correlación Ln_total_pr con mpBERT:", correlacion)
print("P-valor asociado:", p_valor)

Correlación Ln_total_pr con mpBERT: -0.08386335049257175
P-valor asociado: 1.8638283288113262e-07


In [72]:
# Suponiendo que 'base' es tu dataframe y 'variable1' y 'variable2' son las variables que quieres comparar
variable1 = df['ACP_NEG']
variable2 = df['Ln_total_pr']

# Calcular la correlación y el p-valor
correlacion, p_valor = pearsonr(variable1, variable2)

print("Correlación Ln_total_pr con ACP_NEG:", correlacion)
print("P-valor asociado:", p_valor)

Correlación Ln_total_pr con ACP_NEG: -0.1005124497652965
P-valor asociado: 4.050570276433482e-10


In [73]:
# Suponiendo que 'base' es tu dataframe y 'variable1' y 'variable2' son las variables que quieres comparar
variable1 = df['ACP_NEU_noPOS']
variable2 = df['Ln_total_pr']

# Calcular la correlación y el p-valor
correlacion, p_valor = pearsonr(variable1, variable2)

print("Correlación Ln_total_pr con ACP_NEU_noPOS:", correlacion)
print("P-valor asociado:", p_valor)

Correlación Ln_total_pr con ACP_NEU_noPOS: -0.23196497293686053
P-valor asociado: 3.17774381893457e-48


## 1. Elaboración del modelo 1 - Carlos

In [10]:
from sklearn.model_selection import train_test_split #para dividir datos en entrenamiento y testeo
import seaborn as sns #para hacer gráficos, p. ej. el de distribución de una variable
import matplotlib.pyplot as plt #complemento de sns para gráficos 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #para calcular parámetros del modelo
from scipy import stats

In [47]:
X_ols = df.drop(['Ln_total_pr','ACP_NEU_noPOS','ACP_NEG','mpBERT'], axis=1)  # Todas las columnas excepto 'Ln_total_pr'
y_ols = df['Ln_total_pr']  # Columna 'Ln_total_pr'

In [48]:
# Añadir una constante a X_trainOLS para estimar el intercepto
X_ols = sm.add_constant(X_ols)

# Crear el modelo de regresión lineal
ols = sm.OLS(y_ols, X_ols)

# Ajustar el modelo a los datos
resOLS = ols.fit()

# Imprimir el resumen del modelo
print(resOLS.summary())

                            OLS Regression Results                            
Dep. Variable:            Ln_total_pr   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     850.3
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:37:53   Log-Likelihood:                 606.35
No. Observations:                3852   AIC:                            -1155.
Df Residuals:                    3823   BIC:                            -973.3
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [50]:
y_pred_ols = resOLS.predict(X_ols)

###Ojo que uso los datos de testeo
# Calcular el R^2
r2OLS = r2_score(y_ols, y_pred_ols)

# Calcular el MAE
maeOLS = mean_absolute_error(y_ols, y_pred_ols)

# Calcular el RMSE
rmseOLS = np.sqrt(mean_squared_error(y_ols, y_pred_ols))

# Calcular el RAE
raeOLS = maeOLS / abs(y_ols).mean()

# Calcular el RRSE
rrseOLS = rmseOLS / y_ols.std()

# Imprimir resultados
print("Coefficient of Determination (R2):", r2OLS)
print("Mean Absolute Error (MAE):", maeOLS)
print("Relative Absolute Error (RAE):", raeOLS)
print("Root Relative Squared Error (RRSE):", rrseOLS)
print("Root Mean Squared Error (RMSE):", rmseOLS)

Coefficient of Determination (R2): 0.8616460898195593
Mean Absolute Error (MAE): 0.15484612282921958
Relative Absolute Error (RAE): 0.012375867077535158
Root Relative Squared Error (RRSE): 0.37191127000707497
Root Mean Squared Error (RMSE): 0.20672810628760888


In [51]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calcular el VIF para cada variable independiente
vif = pd.DataFrame()
vif["Variable"] = X_ols.columns ## revisar 'X' ##
vif["VIF"] = [variance_inflation_factor(X_ols.values, i) ## revisar 'X' ##
              for i in range(X_ols.shape[1])] ## revisar 'X' ##

# Imprimir el resultado
print(vif)

                                    Variable         VIF
0                                      const  721.861549
1   Ausencia_singulares_presencia_arm_cocina    1.098543
2                          CT_renta_meda_CPA    1.241919
3                                       DP2e    3.448971
4                                  Dist_CBD2    2.953178
5                              EPC_A_emision    1.125151
6                              EPC_B_emision    1.083731
7                              EPC_C_emision    1.045223
8                              EPC_D_emision    1.095702
9                              EPC_E_emision    1.142805
10                             EPC_F_emision    1.055335
11           Estado_contemporaneidad_calidad    1.335591
12                                   FIRE_pr    3.316405
13                                 H_tamaviv    4.359919
14                     Income_Household_2016    2.416702
15                                Precio_red    1.053624
16                            Y

## 2. Elaboración del modelo 2 - sólo mean-pooling BERT

In [53]:
X_ols2 = df.drop(['Ln_total_pr','ACP_NEU_noPOS','ACP_NEG'], axis=1)
y_ols2 = df['Ln_total_pr']  # Columna 'Ln_total_pr'

# Añadir una constante a X_trainOLS para estimar el intercepto
X_ols2= sm.add_constant(X_ols2)

# Crear el modelo de regresión lineal
ols2 = sm.OLS(y_ols2, X_ols2)

# Ajustar el modelo a los datos
resOLS2 = ols2.fit()

# Imprimir el resumen del modelo
print(resOLS2.summary())

                            OLS Regression Results                            
Dep. Variable:            Ln_total_pr   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     820.9
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:46:07   Log-Likelihood:                 606.65
No. Observations:                3852   AIC:                            -1153.
Df Residuals:                    3822   BIC:                            -965.6
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

`mpBERT` **no resulta significativo**, y **no mejora** ningún parámetro del modelo anterior.

In [56]:
y_pred_ols2 = resOLS2.predict(X_ols2)

###Ojo que uso los datos de testeo
# Calcular el R^2
r2OLS2 = r2_score(y_ols2, y_pred_ols2)

# Calcular el MAE
maeOLS2 = mean_absolute_error(y_ols2, y_pred_ols2)

# Calcular el RMSE
rmseOLS2 = np.sqrt(mean_squared_error(y_ols2, y_pred_ols2))

# Calcular el RAE
raeOLS2 = maeOLS2 / abs(y_ols2).mean()

# Calcular el RRSE
rrseOLS2 = rmseOLS2 / y_ols2.std()

# Imprimir resultados
print("Coefficient of Determination (R2):", r2OLS2)
print("Mean Absolute Error (MAE):", maeOLS2)
print("Relative Absolute Error (RAE):", raeOLS2)
print("Root Relative Squared Error (RRSE):", rrseOLS2)
print("Root Mean Squared Error (RMSE):", rmseOLS2)

Coefficient of Determination (R2): 0.8616677395904545
Mean Absolute Error (MAE): 0.15479758221793347
Relative Absolute Error (RAE): 0.01237198753478547
Root Relative Squared Error (RRSE): 0.3718821703278844
Root Mean Squared Error (RMSE): 0.2067119311349373


In [55]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calcular el VIF para cada variable independiente
vif = pd.DataFrame()
vif["Variable"] = X_ols2.columns ## revisar 'X' ##
vif["VIF"] = [variance_inflation_factor(X_ols2.values, i) ## revisar 'X' ##
              for i in range(X_ols2.shape[1])] ## revisar 'X' ##

# Imprimir el resultado
print(vif)

                                    Variable         VIF
0                                      const  836.457875
1   Ausencia_singulares_presencia_arm_cocina    1.098602
2                          CT_renta_meda_CPA    1.243686
3                                       DP2e    3.448974
4                                  Dist_CBD2    2.953295
5                              EPC_A_emision    1.126509
6                              EPC_B_emision    1.083737
7                              EPC_C_emision    1.045331
8                              EPC_D_emision    1.098024
9                              EPC_E_emision    1.153282
10                             EPC_F_emision    1.060224
11           Estado_contemporaneidad_calidad    1.335605
12                                   FIRE_pr    3.333632
13                                 H_tamaviv    4.362522
14                     Income_Household_2016    2.418961
15                                Precio_red    1.054128
16                            Y

## 3. Elaboración del modelo 3 - sólo análisis de sentimiento

### 3.1. `ACP_NEG` y `ACP_NEU_noPOS`

In [86]:
X_ols3 = df.drop(['Ln_total_pr','mpBERT'], axis=1)
y_ols3 = df['Ln_total_pr']  # Columna 'Ln_total_pr'

# Añadir una constante a X_trainOLS para estimar el intercepto
X_ols3= sm.add_constant(X_ols3)

# Crear el modelo de regresión lineal
ols3 = sm.OLS(y_ols3, X_ols3)

# Ajustar el modelo a los datos
resOLS3 = ols3.fit()

# Imprimir el resumen del modelo
print(resOLS3.summary())

                            OLS Regression Results                            
Dep. Variable:            Ln_total_pr   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     806.9
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:30:07   Log-Likelihood:                 634.86
No. Observations:                3852   AIC:                            -1208.
Df Residuals:                    3821   BIC:                            -1014.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [87]:
y_pred_ols3 = resOLS3.predict(X_ols3)

###Ojo que uso los datos de testeo
# Calcular el R^2
r2OLS3 = r2_score(y_ols3, y_pred_ols3)

# Calcular el MAE
maeOLS3 = mean_absolute_error(y_ols3, y_pred_ols3)

# Calcular el RMSE
rmseOLS3 = np.sqrt(mean_squared_error(y_ols3, y_pred_ols3))

# Calcular el RAE
raeOLS3 = maeOLS3 / abs(y_ols3).mean()

# Calcular el RRSE
rrseOLS3 = rmseOLS3 / y_ols3.std()

# Imprimir resultados
print("Coefficient of Determination (R2):", r2OLS3)
print("Mean Absolute Error (MAE):", maeOLS3)
print("Relative Absolute Error (RAE):", raeOLS3)
print("Root Relative Squared Error (RRSE):", rrseOLS3)
print("Root Mean Squared Error (RMSE):", rmseOLS3)

Coefficient of Determination (R2): 0.8636790484530849
Mean Absolute Error (MAE): 0.15409740452777476
Relative Absolute Error (RAE): 0.01231602677925776
Root Relative Squared Error (RRSE): 0.3691687444676061
Root Mean Squared Error (RMSE): 0.2052036644195014


Revisamos el VIF.

In [59]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calcular el VIF para cada variable independiente
vif = pd.DataFrame()
vif["Variable"] = X_ols3.columns ## revisar 'X' ##
vif["VIF"] = [variance_inflation_factor(X_ols3.values, i) ## revisar 'X' ##
              for i in range(X_ols3.shape[1])] ## revisar 'X' ##

# Imprimir el resultado
print(vif)

                                    Variable         VIF
0                                      const  726.056026
1   Ausencia_singulares_presencia_arm_cocina    1.099526
2                          CT_renta_meda_CPA    1.245983
3                                       DP2e    3.450488
4                                  Dist_CBD2    2.953920
5                              EPC_A_emision    1.125977
6                              EPC_B_emision    1.084710
7                              EPC_C_emision    1.045541
8                              EPC_D_emision    1.096300
9                              EPC_E_emision    1.148649
10                             EPC_F_emision    1.056774
11           Estado_contemporaneidad_calidad    1.347712
12                                   FIRE_pr    3.332679
13                                 H_tamaviv    4.362291
14                     Income_Household_2016    2.422888
15                                Precio_red    1.053927
16                            Y

### 3.2. `ACP_NEU_noPOS`

Dado que sólo esta variable tiene relación significativa, pruebo cómo quedaría el modelo.

In [84]:
X_ols4 = df.drop(['Ln_total_pr','mpBERT','ACP_NEG'], axis=1)
y_ols4 = df['Ln_total_pr']  # Columna 'Ln_total_pr'

# Añadir una constante a X_trainOLS para estimar el intercepto
X_ols4= sm.add_constant(X_ols4)

# Crear el modelo de regresión lineal
ols4 = sm.OLS(y_ols4, X_ols4)

# Ajustar el modelo a los datos
resOLS4 = ols4.fit()

# Imprimir el resumen del modelo
print(resOLS4.summary())

                            OLS Regression Results                            
Dep. Variable:            Ln_total_pr   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     834.8
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:29:38   Log-Likelihood:                 634.40
No. Observations:                3852   AIC:                            -1209.
Df Residuals:                    3822   BIC:                            -1021.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [88]:
y_pred_ols4 = resOLS4.predict(X_ols4)

###Ojo que uso los datos de testeo
# Calcular el R^2
r2OLS4 = r2_score(y_ols4, y_pred_ols4)

# Calcular el MAE
maeOLS4 = mean_absolute_error(y_ols4, y_pred_ols4)

# Calcular el RMSE
rmseOLS4 = np.sqrt(mean_squared_error(y_ols4, y_pred_ols4))

# Calcular el RAE
raeOLS4 = maeOLS4 / abs(y_ols4).mean()

# Calcular el RRSE
rrseOLS4 = rmseOLS4 / y_ols4.std()

# Imprimir resultados
print("Coefficient of Determination (R2):", r2OLS4)
print("Mean Absolute Error (MAE):", maeOLS4)
print("Relative Absolute Error (RAE):", raeOLS4)
print("Root Relative Squared Error (RRSE):", rrseOLS4)
print("Root Mean Squared Error (RMSE):", rmseOLS4)

Coefficient of Determination (R2): 0.8636459950734368
Mean Absolute Error (MAE): 0.15411759189969557
Relative Absolute Error (RAE): 0.012317640227673335
Root Relative Squared Error (RRSE): 0.36921349743827203
Root Mean Squared Error (RMSE): 0.20522854050587636


Ya no reviso el VIF, pues no tendría sentido que haya aumentado.

## 4. Elaboración del modelo 4 - BERT y análisis de sentimiento

In [68]:
X_ols5 = df.drop(['Ln_total_pr'], axis=1)
y_ols5 = df['Ln_total_pr']  # Columna 'Ln_total_pr'

# Añadir una constante a X_trainOLS para estimar el intercepto
X_ols5= sm.add_constant(X_ols5)

# Crear el modelo de regresión lineal
ols5 = sm.OLS(y_ols5, X_ols5)

# Ajustar el modelo a los datos
resOLS5 = ols5.fit()

# Imprimir el resumen del modelo
print(resOLS5.summary())

                            OLS Regression Results                            
Dep. Variable:            Ln_total_pr   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     780.8
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:02:27   Log-Likelihood:                 635.09
No. Observations:                3852   AIC:                            -1206.
Df Residuals:                    3820   BIC:                            -1006.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [70]:
y_pred_ols5 = resOLS5.predict(X_ols5)

###Ojo que uso los datos de testeo
# Calcular el R^2
r2OLS5 = r2_score(y_ols5, y_pred_ols5)

# Calcular el MAE
maeOLS5 = mean_absolute_error(y_ols5, y_pred_ols5)

# Calcular el RMSE
rmseOLS5 = np.sqrt(mean_squared_error(y_ols5, y_pred_ols5))

# Calcular el RAE
raeOLS5 = maeOLS5 / abs(y_ols5).mean()

# Calcular el RRSE
rrseOLS5 = rmseOLS5 / y_ols5.std()

# Imprimir resultados
print("Coefficient of Determination (R2):", r2OLS5)
print("Mean Absolute Error (MAE):", maeOLS5)
print("Relative Absolute Error (RAE):", raeOLS5)
print("Root Relative Squared Error (RRSE):", rrseOLS5)
print("Root Mean Squared Error (RMSE):", rmseOLS5)

Coefficient of Determination (R2): 0.8636952341238748
Mean Absolute Error (MAE): 0.15412919759419133
Relative Absolute Error (RAE): 0.012318567797119684
Root Relative Squared Error (RRSE): 0.36914682772968027
Root Mean Squared Error (RMSE): 0.2051914819284268


In [69]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calcular el VIF para cada variable independiente
vif = pd.DataFrame()
vif["Variable"] = X_ols5.columns ## revisar 'X' ##
vif["VIF"] = [variance_inflation_factor(X_ols5.values, i) ## revisar 'X' ##
              for i in range(X_ols5.shape[1])] ## revisar 'X' ##

# Imprimir el resultado
print(vif)

                                    Variable         VIF
0                                      const  848.831873
1   Ausencia_singulares_presencia_arm_cocina    1.099527
2                          CT_renta_meda_CPA    1.247994
3                                       DP2e    3.450488
4                                  Dist_CBD2    2.954161
5                              EPC_A_emision    1.127329
6                              EPC_B_emision    1.084735
7                              EPC_C_emision    1.045634
8                              EPC_D_emision    1.098240
9                              EPC_E_emision    1.157193
10                             EPC_F_emision    1.061866
11           Estado_contemporaneidad_calidad    1.348353
12                                   FIRE_pr    3.357369
13                                 H_tamaviv    4.364570
14                     Income_Household_2016    2.426880
15                                Precio_red    1.054592
16                            Y

Ningún VIF va por arriba de 5.

# Comparación de modelos

In [90]:
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Crear listas para almacenar los resultados de cada modelo
r2_values = []
mae_values = []
aic_values = []
bic_values = []

# Iterar sobre cada modelo
for i in range(2, 6):  # Iterar desde 2 hasta 5 para los modelos ols2, ols3, ..., ols5
    # Obtener X y y para el modelo actual
    X_ols = globals()['X_ols{}'.format(i)]
    y_ols = globals()['y_ols{}'.format(i)]
    
    # Añadir una constante a X_trainOLS para estimar el intercepto
    X_ols = sm.add_constant(X_ols)

    # Crear el modelo de regresión lineal
    ols = sm.OLS(y_ols, X_ols)

    # Ajustar el modelo a los datos
    resOLS = ols.fit()
    
    # Guardar los valores de interés
    r2_values.append(r2_score(y_ols, resOLS.predict(X_ols)))
    mae_values.append(mean_absolute_error(y_ols, resOLS.predict(X_ols)))
    aic_values.append(resOLS.aic)
    bic_values.append(resOLS.bic)

# Crear un DataFrame con los resultados
results_df = pd.DataFrame({
    'Model': ['OLS{}'.format(i) for i in range(2, 6)],
    'R^2': r2_values,
    'MAE': mae_values,
    'AIC': aic_values,
    'BIC': bic_values
})


# Concatenar el DataFrame del modelo OLS con el DataFrame de resultados
results_df = pd.concat([ols_results_df, results_df]).reset_index(drop=True)


# Lista de descripciones para cada modelo
descriptions = [
    'Modelo inicial - Carlos',
    'Sólo BERT',
    'ACP_NEG y ACP_NEU_noPOS',
    'Sólo ACP_NEU_noPOS',
    'BERT y análisis de sentimiento'
]

# Añadir la columna Descripción al DataFrame
results_df['Descripción'] = descriptions

# Imprimir el DataFrame
print(results_df)

  Model       R^2       MAE          AIC          BIC  \
0   OLS  0.861646  0.154846 -1206.182705 -1005.979576   
1  OLS2  0.861668  0.154798 -1153.307265  -965.616832   
2  OLS3  0.863679  0.154097 -1207.725321 -1013.778540   
3  OLS4  0.863646  0.154118 -1208.791450 -1021.101017   
4  OLS5  0.863695  0.154129 -1206.182705 -1005.979576   

                      Descripción  
0         Modelo inicial - Carlos  
1                       Sólo BERT  
2         ACP_NEG y ACP_NEU_noPOS  
3              Sólo ACP_NEU_noPOS  
4  BERT y análisis de sentimiento  


El que más aumenta el R$^2$ y más reduce el MAE (Error Medio Absoluto) es el modelo que considera sólo el análisis de sentimiento, con `ACP_NEG` y `ACP_NEU_noPOS`. Aún así, el impacto es bastante limitado.

Sin embargo, el que tiene menor AIC y BIC (que miden el balance entre capacidad predictiva y complejidad) es el modelo que toma en cuenta sólo a `ACP_NEG`, que es la única del análisis de sentimiento con coeficiente significativo dentro de los modelos..